<a href="https://colab.research.google.com/github/akram01Br/PixelHawk-OD/blob/main/Movie_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 48.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=129dea81e0915d136e9ec6d3ca684fec799bbb3d0b87b21d02ec884e6a76faba
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from math import sqrt
conf =SparkConf().setMaster("local[*]").setAppName("Movie similarity")
sc = SparkContext(conf=conf)

21/12/27 22:51:34 WARN Utils: Your hostname, cj-X550JX resolves to a loopback address: 127.0.1.1; using 192.168.1.101 instead (on interface wlp3s0)
21/12/27 22:51:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/27 22:51:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
def loadMovieNames():
  nameDict={}
  with open("DataBase/u.item") as f:
    for line in f:
      field=line.split("|")
      nameDict[field[0]]=field[1]
  return nameDict


In [ ]:
def makePairs(data):
  user,ratings=data
  (movie1,rating1)=ratings[0]
  (movie2,rating2)=ratings[1]
  return ((movie1,movie2),(rating1,rating2))

In [ ]:
def dropDuplicates(duplicatedData):
  userId,ratings=duplicatedData
  (movie1,rating1)=ratings[0]
  (movie2,rating2)=ratings[1]
  return movie1<movie2

In [ ]:
lines=sc.textFile("file:///home/cj/sparkWorkSpace/DataBase/u.data")

In [ ]:
mappedData=lines.map(lambda x:x.split()).map(lambda x:(int(x[0]),(float(x[1]),float(x[2]))))

In [ ]:
joinedData=mappedData.join(mappedData)

In [ ]:
uniqueData=joinedData.filter(dropDuplicates)

In [ ]:
moviepairs=uniqueData.map(makePairs)

In [ ]:
makePairsRatings=moviepairs.groupByKey()

In [ ]:
def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

In [ ]:
moviePairSimilarity= makePairsRatings.mapValues(computeCosineSimilarity).cache()

In [ ]:
import sys

#if (len(sys.argv)>1):
scoreThresholds=0.97
coOccurenceThreshold = 50
#movieID=int(sys.argv[1])
movieID=253
filterdResults=moviePairSimilarity.filter(lambda pair,sim :\
 (pair[0]==movieID or pair[1]==movieID)\
 (sim[0] > scoreThresholds and sim[1] > coOccurenceThreshold )
    )
results=filterdResults.map(lambda pair,sim:(sim,pair)).sortByKey(ascending=False).take(10)
print("Top 10 similar movies for " + nameDict[movieID])
for result in results:
    (sim, pair) = result
    # Display the similarity result that isn't the movie we're looking at
    similarMovieID = pair[0]
    if (similarMovieID == movieID):
        similarMovieID = pair[1]
    print(nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1]))


21/12/27 22:52:05 ERROR Executor: Exception in task 1.0 in stage 2.0 (TID 9)/ 4]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 595, in process
    out_iter = func(split_index, iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 425, in func
    return f(iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 1141, in <lambda>
    return self.mapPartitions(lambda i: [sum(1 for _ in i)]).sum()
  File "/opt/spark/python/pyspark/rd

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 2.0 failed 1 times, most recent failure: Lost task 1.0 in stage 2.0 (TID 9, 192.168.1.101, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 595, in process
    out_iter = func(split_index, iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rd

21/12/27 22:52:05 WARN BlockManager: Block rdd_13_0 could not be removed as it was not found on disk or in memory
21/12/27 22:52:05 WARN TaskSetManager: Lost task 0.0 in stage 2.0 (TID 8, 192.168.1.101, executor driver): TaskKilled (Stage cancelled)
21/12/27 22:52:05 WARN BlockManager: Putting block rdd_13_2 failed due to exception org.apache.spark.TaskKilledException.
21/12/27 22:52:05 WARN BlockManager: Block rdd_13_2 could not be removed as it was not found on disk or in memory
21/12/27 22:52:05 WARN TaskSetManager: Lost task 2.0 in stage 2.0 (TID 10, 192.168.1.101, executor driver): TaskKilled (Stage cancelled)


In [ ]:
sc.stop()